In [1]:
import numpy as np

In [2]:
def build_graph(fname):
    graph = dict()
    with open(fname) as infile:
        count = 0
        for line in infile:
            line = line.strip()
            if (count == 1):
                # nodes = line.split(',')
                nodes = line.split(';')
                for n in nodes:
                    graph.update({ n : [] })
            if (count > 3):
                if (line == ''):
                    break
                edge = line.split(' ')
                edge = edge[1:]
                # print(tuple(edge))
                if (edge[1] == '-->'):
                    graph[edge[0]].append(edge[2])
                elif (edge[1] == '<--'):
                    graph[edge[2]].append(edge[0])

            count += 1

    return graph

def ancestor(n1, n2, g):
    descendents = [n1]

    old = 0
    new = 1 
    while (old != new):
        for i in range(old, new):
            updates = g[descendents[i]]
            for n in g[descendents[i]]:
                if n in descendents:
                    updates.remove(n)
            descendents.extend(updates)

        # print(descendents)
        old = new
        new = len(descendents)

    return n2 in descendents

In [3]:
def load_edges(fname):
    rev_map = dict(
        {
            '-->' : '<--',
            '<--' : '-->',
            '<->' : '<->',
            'o->' : '<-o',
            '<-o' : 'o->',
            'o-o' : 'o-o',
            '---' : '---'
        }
    )

    print('loading edges from ' + fname)

    eef = open(fname, 'r')

    eeflines = eef.readlines()

    edges = list()

    linum = 0
    for line in eeflines:
        if (linum > 3):
            if line.strip()=='':
                break
            if line.strip()=='Ambiguous triples (i.e. list of triples for which there is ambiguous data about whether they are colliders or not):':
                break
            # print(line)
            temp = line.strip().split(' ')
            edge = [temp[1], temp[2], temp[3]]
            # edge.sort()
            # print(tuple(edge))
            if (edge[0] < edge[2]):
                edges.append(tuple(edge))
            else:
                edge.reverse()
                edge[1] = rev_map[edge[1]]
                edges.append(tuple(edge))
                
        linum+=1
        
    edges.sort()

    return edges

In [4]:
def calc_markov_equiv_arrow_pr(exp, true):
    tp = np.zeros(7)
    fp = np.zeros(7)
    fn = np.zeros(7)
    tn = np.zeros(7)
    
    for e1 in exp:
        # print(e1)
        for e2 in true:
            # print(e2)
            if (e1[0] == e2[0]) & (e1[2] == e2[2]):
                idx = 0
                if ('X' in e1[0]) & ('X' in e1[2]):
                    idx = 1
                elif 'X' in e1[0]:
                    idx = 2
                elif 'Y' in e1[0]:
                    idx = 3
                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                    idx = 4
                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                    idx = 5
                for end in [0, 2]:
                    if e1[1][end] == '-':
                        if e2[1][end] != '-':
                            fn[0] += 1
                            fn[idx] += 1
                        else:
                            tn[0] += 1
                            tn[idx] += 1
                    elif (e1[1][end] == '>') | (e1[1][end] == '<'):
                        if e2[1][end] == '-':
                            fp[0] += 1
                            fp[idx] += 1
                        else:
                            tp[0] += 1
                            tp[idx] += 1
            
    tp[6] = tp[4] + tp[5]
    fp[6] = fp[4] + fp[5]
    fn[6] = fn[4] + fn[5]
    tn[6] = tn[4] + tn[5]
    
#     print(tp)
#     print(fp)
#     print(fn)
#     print(tn)
    
    return tp/(tp+fp), tp/(tp+fn), (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))

In [5]:
def calc_markov_equiv_shd(exp, true, numVars):
    missing = np.zeros(7)
    added = np.zeros(7)
    misoriented = np.zeros(7)
    
    for e1 in exp:
        found = False
        for e2 in true:
            # print(e2)
            if (e1[0] == e2[0]) & (e1[2] == e2[2]):
                found = True
                idx = 0
                if ('X' in e1[0]) & ('X' in e1[2]):
                    idx = 1
                elif 'X' in e1[0]:
                    idx = 2
                elif 'Y' in e1[0]:
                    idx = 3
                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                    idx = 4
                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                    idx = 5
                if (e1[1] != e2[1]):
                    misoriented[0] += 1
                    misoriented[idx] += 1
                break
                
        if (not found):
            added[0] += 1
            idx = 0
            if ('X' in e1[0]) & ('X' in e1[2]):
                idx = 1
            elif 'X' in e1[0]:
                idx = 2
            elif 'Y' in e1[0]:
                idx = 3
            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                idx = 4
            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                idx = 5
            added[idx] += 1
            
    for e1 in true:
        found = False
        for e2 in exp:
            # print(e2)
            if (e1[0] == e2[0]) & (e1[2] == e2[2]):
                found = True
                break
                
        if (not found):
            missing[0] += 1
            idx = 0
            if ('X' in e1[0]) & ('X' in e1[2]):
                idx = 1
            elif 'X' in e1[0]:
                idx = 2
            elif 'Y' in e1[0]:
                idx = 3
            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                idx = 4
            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                idx = 5
            missing[idx] += 1
            
    missing[6] = missing[4] + missing[5]
    added[6] = added[4] + added[5]
    misoriented[6] = misoriented[4] + misoriented[5]
    
    numEdges = np.zeros(7)
    totVars = np.sum(numVars)
    numEdges[0] = totVars*(totVars-1) / 2
    numEdges[1] = numVars[0]*(numVars[0]-1) / 2
    numEdges[2] = numVars[0]*numVars[1]
    numEdges[3] = numVars[1]*(numVars[1]-1) / 2
    numEdges[4] = numVars[0]*numVars[2]
    numEdges[5] = numVars[1]*numVars[2]
    numEdges[6] = (numVars[0]+numVars[1])*numVars[2]
#     print(missing)
#     print(added)
#     print(misoriented)
    
    return (missing + added + misoriented) / numEdges

In [6]:
def calc_pag_arrow_pr(exp, pag, true): # true should be a graph from build_graph instead of a list of edges from load_edges so ancestry can be checked
    tp = np.zeros(7)
    fp = np.zeros(7)
    fn = np.zeros(7)

    for e1 in exp:
        for e2 in pag:
            if (e1[0] == e2[0]) & (e1[2] == e2[2]):
                for end in [0, 2]:
                    if e2[1][end] == 'o':
                        if e1[1][end] == 'o':
                            tp[0] += 1
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                tp[1] += 1
                            elif 'X' in e1[0]:
                                tp[2] += 1
                            elif 'Y' in e1[0]:
                                tp[3] +=1
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                tp[4] += 1
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                tp[5] += 1
                        elif (e1[1][end] == '>') | (e1[1][end] == '<'):
                            if not ancestor(e1[end], e1[(end+2)%4], true):
                                tp[0] += 1
                                if ('X' in e1[0]) & ('X' in e1[2]):
                                    tp[1] += 1
                                elif 'X' in e1[0]:
                                    tp[2] += 1
                                elif 'Y' in e1[0]:
                                    tp[3] +=1
                                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                    tp[4] += 1
                                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                    tp[5] += 1
                            else:
                                fn[0] += 1
                                if ('X' in e1[0]) & ('X' in e1[2]):
                                    fn[1] += 1
                                elif 'X' in e1[0]:
                                    fn[2] += 1
                                elif 'Y' in e1[0]:
                                    fn[3] +=1
                                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                    fn[4] += 1
                                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                    fn[5] += 1
                        elif (e1[1][end] == '-'):
                            if ancestor(e1[end], e1[(end+2)%4], true):
                                tp[0] += 1
                                if ('X' in e1[0]) & ('X' in e1[2]):
                                    tp[1] += 1
                                elif 'X' in e1[0]:
                                    tp[2] += 1
                                elif 'Y' in e1[0]:
                                    tp[3] +=1
                                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                    tp[4] += 1
                                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                    tp[5] += 1
                            else:
                                fp[0] += 1
                                if ('X' in e1[0]) & ('X' in e1[2]):
                                    fp[1] += 1
                                elif 'X' in e1[0]:
                                    fp[2] += 1
                                elif 'Y' in e1[0]:
                                    fp[3] +=1
                                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                    fp[4] += 1
                                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                    fp[5] += 1
                    elif (e2[1][end] == '>') | (e2[1][end] == '<'):
                        if (e1[1][end] == 'o'):
                            fp[0] += 0.5
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                fp[1] += 0.5
                            elif 'X' in e1[0]:
                                fp[2] += 0.5
                            elif 'Y' in e1[0]:
                                fp[3] +=0.5
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                fp[4] += 0.5
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                fp[5] += 0.5
                        elif (e1[1][end] == '>') | (e1[1][end] == '<'):
                            tp[0] += 1
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                tp[1] += 1
                            elif 'X' in e1[0]:
                                tp[2] += 1
                            elif 'Y' in e1[0]:
                                tp[3] +=1
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                tp[4] += 1
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                tp[5] += 1
                        elif (e1[1][end] == '-'):
                            fp[0] += 1
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                fp[1] += 1
                            elif 'X' in e1[0]:
                                fp[2] += 1
                            elif 'Y' in e1[0]:
                                fp[3] +=1
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                fp[4] += 1
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                fp[5] += 1
                    elif (e2[1][end] == '-'):
                        if (e1[1][end] == 'o'):
                            fn[0] += 0.5
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                fn[1] += 0.5
                            elif 'X' in e1[0]:
                                fn[2] += 0.5
                            elif 'Y' in e1[0]:
                                fn[3] += 0.5
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                fn[4] += 0.5
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                fn[5] += 0.5
                        elif (e1[1][end] == '>') | (e1[1][end] == '<'):
                            tp[0] += 1
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                tp[1] += 1
                            elif 'X' in e1[0]:
                                tp[2] += 1
                            elif 'Y' in e1[0]:
                                tp[3] +=1
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                tp[4] += 1
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                tp[5] += 1
                        elif (e1[1][end] == '-'):
                            fn[0] += 1
                            if ('X' in e1[0]) & ('X' in e1[2]):
                                fn[1] += 1
                            elif 'X' in e1[0]:
                                fn[2] += 1
                            elif 'Y' in e1[0]:
                                fn[3] +=1
                            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                                fn[4] += 1
                            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                                fn[5] += 1
    tp[6] = tp[4] + tp[5]
    fp[6] = fp[4] + fp[5]
    fn[6] = fn[4] + fn[5]
    
    return tp/(tp+fp), tp/(tp+fn)

In [7]:
def calc_adj_pr(exp, true, numVars):
    tp = np.zeros(7)
    fp = np.zeros(7)
    fn = np.zeros(7)
    tn = 0.5 * numVars * (numVars-1) * np.ones(7)
    
    for e1 in true:
        idx = 0
        if ('X' in e1[0]) & ('X' in e1[2]):
            idx = 1
        elif 'X' in e1[0]:
            idx = 2
        elif 'Y' in e1[0]:
            idx = 3
        elif ('Survival' in e1[0]) & ('X' in e1[2]):
            idx = 4
        elif ('Survival' in e1[0]) & ('Y' in e1[2]):
            idx = 5
        fn[0] += 1
        fn[idx] +=1

    for e1 in exp:
        real = False
        for e2 in true:
            if (e1[0] == e2[0]) & (e1[2] == e2[2]):
                real = True
                idx = 0
                if ('X' in e1[0]) & ('X' in e1[2]):
                    idx = 1
                elif 'X' in e1[0]:
                    idx = 2
                elif 'Y' in e1[0]:
                    idx = 3
                elif ('Survival' in e1[0]) & ('X' in e1[2]):
                    idx = 4
                elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                    idx = 5
                tp[0] += 1
                tp[idx] +=1
                break
        if (not real):
            idx = 0
            if ('X' in e1[0]) & ('X' in e1[2]):
                idx = 1
            elif 'X' in e1[0]:
                idx = 2
            elif 'Y' in e1[0]:
                idx = 3
            elif ('Survival' in e1[0]) & ('X' in e1[2]):
                idx = 4
            elif ('Survival' in e1[0]) & ('Y' in e1[2]):
                idx = 5
            fp[0] += 1
            fp[idx] +=1
                
    fn -= tp
    tn -= (tp + fp + fn)
            
    tp[6] = tp[4] + tp[5]
    fp[6] = fp[4] + fp[5]
    fn[6] = fn[4] + fn[5]
    tn[6] = tn[4] + tn[5]
    
#     print(tp)
#     print(fp)
#     print(fn)
#     print(tn)
    
    return tp/(tp+fp), tp/(tp+fn), (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))

In [13]:
true = build_graph('graph/graph.n1000.p100.txt')
pag = load_edges('graph/pag.n1000.p100.txt')
fcim = load_edges('../fcimax_test.txt')
# mgm = load_edges('mgm_local.txt')
# cpc = load_edges('cpc_local.txt')
# pcm = load_edges('pcm_local.txt')

loading edges from graph/pag.n1000.p100.txt
loading edges from ../fcimax_test.txt


In [14]:
prec, rec, mcc = calc_adj_pr(fcim, pag, 100)
print('adj precision:\t' + str(np.round(prec[0],3)))
print('adj recall:\t' + str(np.round(rec[0],3)))
print('adj MCC:\t' + str(np.round(mcc[0],3)))

adj precision:	0.984
adj recall:	0.92
adj MCC:	0.949


/home/t-love01/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [15]:
prec, rec = calc_pag_arrow_pr(fcim, pag, true)
print('arrow precision:  ' + str(np.round(prec[0],3)))
print('arrow recall:     ' + str(np.round(rec[0],3)))
# print('adj MCC:\t' + str(np.round(mcc[0],3)))

arrow precision:  0.988
arrow recall:     0.803


/home/t-love01/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:151: RuntimeWarning: invalid value encountered in true_divide


In [8]:
true = build_graph('../../simulations_200327/true_out/sout_pag_0.txt')
# true = load_edges('graph/mec.n1000.p100.txt')
# pcm = load_edges('../pc50_test_5.txt')
pag = load_edges('../../simulations_200327/true_out/sout_pag_0.txt')
fcim = load_edges('../fcim_0_0.5_00500_cens_possParFull.txt')
# mgm = load_edges('mgm_local.txt')
# cpc = load_edges('cpc_local.txt')
# pcm = load_edges('pcm_local.txt')

loading edges from ../../simulations_200327/true_out/sout_pag_0.txt
loading edges from ../fcim_0_0.5_00500_cens_possParFull.txt


In [9]:
prec, rec, mcc = calc_adj_pr(fcim, pag, 110)
print('adj precision:\t' + str(np.round(prec,3)))
print('adj recall:\t' + str(np.round(rec,3)))
print('adj MCC:\t' + str(np.round(mcc,3)))

adj precision:	[0.912 0.962 0.909 0.864 1.    0.5   0.909]
adj recall:	[0.86  0.962 0.806 0.905 1.    0.333 0.833]
adj MCC:	[0.883 0.961 0.855 0.884 1.    0.408 0.87 ]


In [10]:
prec, rec = calc_pag_arrow_pr(fcim, pag, true)
print('arrow precision:  ' + str(np.round(prec,3)))
print('arrow recall:     ' + str(np.round(rec,3)))
# print('adj MCC:\t' + str(np.round(mcc,3)))

arrow precision:  [0.851 0.821 0.832 0.909 0.917 1.    0.929]
arrow recall:     [0.776 0.753 0.802 0.758 0.71  1.    0.743]


In [11]:
shd = calc_markov_equiv_shd(fcim, pag, [50, 50, 10])
print('Normalized SHD:  ' + str(np.round(shd,4)))

Normalized SHD:  [0.0122 0.0098 0.0168 0.0098 0.006  0.008  0.007 ]


In [24]:
true = build_graph('../../simulations_200327/true_out/sout_pag_0.txt')
# true = load_edges('graph/mec.n1000.p100.txt')
# pcm = load_edges('../pc50_test_5.txt')
pag = load_edges('../../simulations_200327/true_out/sout_pag_0.txt')
fcim = load_edges('../cfci_0_0.5_00500_cens_possParFull.txt')
# mgm = load_edges('mgm_local.txt')
# cpc = load_edges('cpc_local.txt')
# pcm = load_edges('pcm_local.txt')

loading edges from ../../simulations_200327/true_out/sout_pag_0.txt
loading edges from ../cfci_0_0.5_00500_cens_possParFull.txt


In [25]:
prec, rec, mcc = calc_adj_pr(fcim, pag, 110)
print('adj precision:\t' + str(np.round(prec,3)))
print('adj recall:\t' + str(np.round(rec,3)))
print('adj MCC:\t' + str(np.round(mcc,3)))

adj precision:	[0.912 0.962 0.909 0.864 1.    0.5   0.909]
adj recall:	[0.86  0.962 0.806 0.905 1.    0.333 0.833]
adj MCC:	[0.883 0.961 0.855 0.884 1.    0.408 0.87 ]


In [26]:
prec, rec = calc_pag_arrow_pr(fcim, pag, true)
print('arrow precision:  ' + str(np.round(prec,3)))
print('arrow recall:     ' + str(np.round(rec,3)))
# print('adj MCC:\t' + str(np.round(mcc,3)))

arrow precision:  [0.783 0.756 0.738 0.906 0.88  1.    0.889]
arrow recall:     [0.735 0.775 0.747 0.696 0.688 0.5   0.667]


In [27]:
shd = calc_markov_equiv_shd(fcim, pag, [50, 50, 10])
print('Normalized SHD:  ' + str(np.round(shd,4)))

Normalized SHD:  [0.0105 0.0098 0.0152 0.0065 0.004  0.006  0.005 ]


In [19]:
# true = build_graph('../fci-max/rCausalMGM/graph/graph.n1000.p100.txt')
# true = load_edges('graph/mec.n1000.p100.txt')
# pcm = load_edges('../pc50_test_5.txt')
true = load_edges('../../simulations_200327/true_out/sout_0.txt')
mgm = load_edges('../fuwmgm_steps_0_00500.txt')
# mgm = load_edges('mgm_local.txt')
# cpc = load_edges('cpc_local.txt')
# pcm = load_edges('pcm_local.txt')
e2rm = []
for e in true:
    if ('Survival' in e[0]):
        e2rm.append(e)
for e in e2rm:
    true.remove(e)

loading edges from ../../simulations_200327/true_out/sout_0.txt
loading edges from ../fuwmgm_steps_0_00500.txt


In [20]:
prec, rec, mcc = calc_adj_pr(mgm, true, 100)
print('adj precision:\t' + str(np.round(prec,3)))
print('adj recall:\t' + str(np.round(rec,3)))
print('adj MCC:\t' + str(np.round(mcc,3)))

adj precision:	[0.715 0.532 0.85  0.739   nan   nan   nan]
adj recall:	[0.853 0.962 0.823 0.81    nan   nan   nan]
adj MCC:	[0.776 0.713 0.834 0.773   nan   nan   nan]


/home/t-love01/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


In [21]:
true

[('X0', '-->', 'X18'),
 ('X0', '<--', 'Y44'),
 ('X1', '-->', 'X2'),
 ('X1', '<--', 'X15'),
 ('X1', '<--', 'X19'),
 ('X1', '<--', 'X27'),
 ('X1', '<--', 'Y22'),
 ('X10', '-->', 'X9'),
 ('X10', '-->', 'Y5'),
 ('X10', '<--', 'Y37'),
 ('X11', '-->', 'X27'),
 ('X11', '<--', 'X31'),
 ('X11', '<--', 'X9'),
 ('X13', '<--', 'X28'),
 ('X13', '<--', 'X49'),
 ('X13', '<--', 'Y49'),
 ('X15', '<--', 'Y1'),
 ('X15', '<--', 'Y43'),
 ('X16', '-->', 'Y46'),
 ('X17', '-->', 'Y17'),
 ('X18', '<--', 'X22'),
 ('X18', '<--', 'Y6'),
 ('X19', '-->', 'X43'),
 ('X19', '<--', 'Y7'),
 ('X2', '-->', 'X23'),
 ('X20', '<--', 'Y18'),
 ('X22', '-->', 'Y17'),
 ('X22', '<--', 'Y31'),
 ('X23', '<--', 'Y0'),
 ('X24', '<--', 'X25'),
 ('X26', '-->', 'Y41'),
 ('X26', '-->', 'Y8'),
 ('X27', '<--', 'Y33'),
 ('X28', '-->', 'X36'),
 ('X29', '-->', 'Y18'),
 ('X3', '-->', 'Y20'),
 ('X3', '<--', 'Y6'),
 ('X30', '-->', 'X9'),
 ('X30', '-->', 'Y23'),
 ('X30', '<--', 'Y10'),
 ('X31', '<--', 'Y7'),
 ('X32', '-->', 'X41'),
 ('X33', '-->'

In [59]:
# true = build_graph('../fci-max/rCausalMGM/graph/graph.n1000.p100.txt')
# true = load_edges('graph/mec.n1000.p100.txt')
# pcm = load_edges('../pc50_test_5.txt')
true = load_edges('../../simulations_200327/true_out/sout_0.txt')
pcm = load_edges('../pcm_0_0.5_00500_cens_possParFull.txt')
# mgm = load_edges('mgm_local.txt')
# cpc = load_edges('cpc_local.txt')
# pcm = load_edges('pcm_local.txt')

loading edges from ../../simulations_200327/true_out/sout_0.txt
loading edges from ../pcm_0_0.5_00500_cens_possParFull.txt


In [60]:
prec, rec, mcc = calc_adj_pr(pcm, true, 110)
print('adj precision:\t' + str(np.round(prec,3)))
print('adj recall:\t' + str(np.round(rec,3)))
print('adj MCC:\t' + str(np.round(mcc,3)))

adj precision:	[0.904 0.962 0.909 0.826 1.    0.5   0.909]
adj recall:	[0.86  0.962 0.806 0.905 1.    0.333 0.833]
adj MCC:	[0.879 0.961 0.855 0.864 1.    0.408 0.87 ]


In [61]:
prec, rec, mcc = calc_markov_equiv_arrow_pr(pcm, true)
print('arrow precision:  ' + str(np.round(prec,3)))
print('arrow recall:     ' + str(np.round(rec,3)))
print('adj MCC:\t' + str(np.round(mcc,3)))

arrow precision:  [0.764 0.81  0.75  0.692 0.8   1.    0.833]
arrow recall:     [0.529 0.68  0.48  0.474 0.444 1.    0.5  ]
adj MCC:	[0.384 0.527 0.343 0.277 0.372 1.    0.436]


In [62]:
shd = calc_markov_equiv_shd(pcm, true, [50, 50, 10])
print('Normalized SHD:  ' + str(np.round(shd,4)))

Normalized SHD:  [0.0128 0.0082 0.0172 0.0131 0.01   0.006  0.008 ]


In [63]:
# true = build_graph('../fci-max/rCausalMGM/graph/graph.n1000.p100.txt')
# true = load_edges('graph/mec.n1000.p100.txt')
# pcm = load_edges('../pc50_test_5.txt')
true = load_edges('../../simulations_200327/true_out/sout_0.txt')
pcm = load_edges('../pcm_0_0.5_00500_cens_possPar.txt')
# mgm = load_edges('mgm_local.txt')
# cpc = load_edges('cpc_local.txt')
# pcm = load_edges('pcm_local.txt')

loading edges from ../../simulations_200327/true_out/sout_0.txt
loading edges from ../pcm_0_0.5_00500_cens_possPar.txt


In [64]:
prec, rec, mcc = calc_adj_pr(pcm, true, 110)
print('adj precision:\t' + str(np.round(prec,3)))
print('adj recall:\t' + str(np.round(rec,3)))
print('adj MCC:\t' + str(np.round(mcc,3)))

adj precision:	[0.904 0.962 0.909 0.826 1.    0.5   0.909]
adj recall:	[0.86  0.962 0.806 0.905 1.    0.333 0.833]
adj MCC:	[0.879 0.961 0.855 0.864 1.    0.408 0.87 ]


In [65]:
prec, rec, mcc = calc_markov_equiv_arrow_pr(pcm, true)
print('arrow precision:  ' + str(np.round(prec,3)))
print('arrow recall:     ' + str(np.round(rec,3)))
print('adj MCC:\t' + str(np.round(mcc,3)))

arrow precision:  [0.629 0.545 0.614 0.765 0.6   1.    0.667]
arrow recall:     [0.538 0.48  0.54  0.684 0.333 1.    0.4  ]
adj MCC:	[0.223 0.081 0.201 0.476 0.124 1.    0.218]


In [66]:
shd = calc_markov_equiv_shd(pcm, true, [50,50,10])
print('Normalized SHD:  ' + str(np.round(shd,4)))

Normalized SHD:  [0.0127 0.0122 0.016  0.0098 0.012  0.006  0.009 ]
